In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [19]:
# https://pytorch.org/vision/stable/datasets.html#fashion-mnist

root_path = "/Users/chiawei/.data"

training_data = datasets.FashionMNIST(
    root=root_path, 
    train=True, 
    download=True,
    transform=ToTensor())

testing_data = datasets.FashionMNIST(
    root=root_path,
    train=False,
    download=True,
    transform=ToTensor())

In [20]:
batch_size = 64

#https://pytorch.org/docs/stable/data.html
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)


#n, c, h, w 
#n = batch size
#c = channel
#h = height
#w = width
for x, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", x.shape)
    print("Shae of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shae of y:  torch.Size([64]) torch.int64


In [21]:
device="cuda" if torch.cuda.is_available() else "cpu"

print("Using {} device".format(device))

Using cpu device


In [22]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10), 
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [23]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [24]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        if batch %100 == 0:
            
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302897  [    0/60000]
loss: 2.300716  [ 6400/60000]
loss: 2.284170  [12800/60000]
loss: 2.276606  [19200/60000]
loss: 2.258866  [25600/60000]
loss: 2.239992  [32000/60000]
loss: 2.241188  [38400/60000]
loss: 2.222671  [44800/60000]
loss: 2.234561  [51200/60000]
loss: 2.184297  [57600/60000]
Test Error: 
 Accuracy: 44.0%, Avg loss: 0.034480 

Epoch 2
-------------------------------
loss: 2.219641  [    0/60000]
loss: 2.211097  [ 6400/60000]
loss: 2.177180  [12800/60000]
loss: 2.172996  [19200/60000]
loss: 2.115947  [25600/60000]
loss: 2.090332  [32000/60000]
loss: 2.099104  [38400/60000]
loss: 2.049855  [44800/60000]
loss: 2.080185  [51200/60000]
loss: 1.984211  [57600/60000]
Test Error: 
 Accuracy: 49.4%, Avg loss: 0.031474 

Epoch 3
-------------------------------
loss: 2.050646  [    0/60000]
loss: 2.023901  [ 6400/60000]
loss: 1.939300  [12800/60000]
loss: 1.959699  [19200/60000]
loss: 1.864656  [25600/60000]
loss: 1.813117  [32000/600

In [26]:
torch.save(model.state_dict(), "model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')